In [ ]:
from Other_To_nparray import generate_data
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from model_evalution import metric_array
from Model.PC_6_10cv_model import c_v
import pandas as pd
import numpy as np
import os


output_path = 'Output/Embedding/10cv'
train_data, train_labels = generate_data('Train_Postive_3175.fasta', 'Train_Negative_3175.fasta',method='integer')

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import CuDNNLSTM,Add,Input
from keras.layers import LeakyReLU
from keras import models
from keras import optimizers
import os
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES']='2'
def create_network():
    model = models.Sequential()
    # Input - Layer
    model.add(Embedding(21, 64, input_length=200))
    model.add(Conv1D(64,16 ,activation = "relu", padding="same",input_shape=(200, 64)))
    # Hidden - Layers
    lstm = CuDNNLSTM(units=100,return_sequences=False)
    model.add(lstm)
    # Output- Layer
    model.add(Dense(1, activation = "sigmoid"))

    model.compile(optimizer=optimizers.Adam(lr=7*1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
e_s = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          min_delta=0,
                                          patience=80,
                                          verbose=0, mode='min')
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=300, batch_size=int(0.5*len(train_data)),
                                 verbose=0)

In [ ]:
r=cross_val_score(neural_network, train_data ,train_labels, cv=10,scoring='accuracy',fit_params = {'callbacks': [e_s]})